#  1. Manage bdd

In [52]:
import os
import pandas as pd


path_to_datafiles = os.path.join(os.getcwd(), 'datafiles')

test = pd.read_csv(os.path.join(path_to_datafiles, 'BD_71_test.csv'),header =None)
train = pd.read_csv(os.path.join(path_to_datafiles, 'BD_71_train.csv'),header =None)
val = pd.read_csv(os.path.join(path_to_datafiles, 'BD_71_val.csv'),header =None)


In [53]:
test.head()

,0,1,2,3,4,5,6,7
0,BD_71/Anthidium septemspinosum/Anthidium septe...,20.089286,51.250000,313.75,298.50,Anthophila,500,336
1,BD_71/Bombus pascuorum/Bombus pascuorum54764.jpeg,48.750000,46.250000,483.75,408.75,Anthophila,500,438
2,BD_71/Nomada goodeniana/Nomada goodeniana5899.jpg,5.000000,16.250000,418.75,320.00,Anthophila,500,333
3,BD_71/Apis mellifera/Apis mellifera11416.jpg,444.000000,210.000000,696.00,520.00,Anthophila,800,602
4,BD_71/Andrena gravida/dsc_9352_21avr20.jpg,37.946429,2.026786,451.25,445.00,Anthophila,500,454


In [54]:
!mkdir BD_71
!mkdir BD_71/train
!mkdir BD_71/val
!mkdir BD_71/test

!mkdir BD_71/train/Anthophila
!mkdir BD_71/val/Anthophila
!mkdir BD_71/test/Anthophila

In [55]:
bd_71_location = '/home/basile/Documents/projet_bees_detection_basile/data_bees_detection'
bd_71_new_location = os.getcwd()

In [56]:
test['img_name'] = test[0].apply(lambda x: x.split('/')[-1])
test['new_path'] = test['img_name'].apply(lambda x: os.path.join(bd_71_new_location, 'BD_71', 'test','Anthophila', x))
test['old_path'] = test[0].apply(lambda x: os.path.join(bd_71_location, x))

train['img_name'] = train[0].apply(lambda x: x.split('/')[-1])
train['new_path'] = train['img_name'].apply(lambda x: os.path.join(bd_71_new_location, 'BD_71', 'train','Anthophila', x))
train['old_path'] = train[0].apply(lambda x: os.path.join(bd_71_location, x))

val['img_name'] = val[0].apply(lambda x: x.split('/')[-1])
val['new_path'] = val['img_name'].apply(lambda x: os.path.join(bd_71_new_location, 'BD_71', 'val','Anthophila', x))
val['old_path'] = val[0].apply(lambda x: os.path.join(bd_71_location, x))


In [57]:

def copy_image(df):

    for i in range(len(df)):

        # copy image
        cmd = 'cp "'+ df['old_path'][i]+'" "'+ df['new_path'][i] + '"'
        os.system(cmd)

In [58]:
copy_image(test)
copy_image(train)
copy_image(val)

# 2 Create label files.txt

In [150]:
! mkdir datasets/BD_71/labels/train
! mkdir datasets/BD_71/labels/test
! mkdir datasets/BD_71/labels/val

! mkdir datasets/BD_71/labels/train/Anthophila
! mkdir datasets/BD_71/labels/test/Anthophila
! mkdir datasets/BD_71/labels/val/Anthophila

mkdir: cannot create directory ‘datasets/BD_71/labels/train’: File exists
mkdir: cannot create directory ‘datasets/BD_71/labels/test’: File exists
mkdir: cannot create directory ‘datasets/BD_71/labels/val’: File exists
mkdir: cannot create directory ‘datasets/BD_71/labels/val/Anthophila’: File exists


## 2.1 Test for test split

In [151]:
import pandas as pd
import os

path_to_jsons = '../projet_bees_detection_basile/data_bees_detection/BD_71_Annotations/JSON'

df_test = pd.read_csv(os.path.join(os.getcwd(), 'datafiles', 'BD_71_test.csv'),header =None)



In [152]:
df_test['real_label'] = df_test[0].apply(lambda x: x.split('/')[-2])
df_test['img_name'] = df_test[0].apply(lambda x: x.split('/')[-1])

In [153]:
import json

def get_json_annotations(img_name,real_label):
    
   json_name = real_label+'.json' 
   json_path = os.path.join(path_to_jsons, json_name)

   img_path = './BD_71/'+ real_label + '/' + img_name

   # load json and serch for file_paths = img_name

   f = open(json_path)
   data = json.load(f)

   for img in data:

      if img['file_path'] == img_path :

         return list(img['boxes'])

In [154]:
def img_name_to_txt_path(split, img_name):
    
    ext = img_name.split('.')[-1]
    txt = img_name.replace(ext, '.txt')

    return os.path.join( './datasets/BD_71/labels', split, 'Anthophila', txt)


In [155]:
df_test['txt_path'] = df_test['img_name'].apply(lambda x: img_name_to_txt_path('test', x))

In [156]:
def write_txt_file(boxes, txt_path):
    
      
      with open(txt_path, 'w') as f:
         
         for box in boxes:

            xmin = box['xmin']
            ymin = box['ymin']
            xmax = box['xmax']
            ymax = box['ymax']




            x_center = (xmin + xmax) / 2
            y_center = (ymin + ymax) / 2
            w = xmax - xmin
            h = ymax - ymin


            line = '0 ' + str(x_center) + ' ' + str(y_center) + ' ' + str(w) + ' ' + str(h) + '\n'
               
            f.write(line)
               
      return txt_path


In [157]:
# iterate over all rows of df_test

for i in range(len(df_test)):

    boxes = get_json_annotations(df_test['img_name'][i], df_test['real_label'][i])
    txt_path = df_test['txt_path'][i]
    
    write_txt_file(boxes, txt_path)

## 2.1 Train

In [158]:

df_train = pd.read_csv(os.path.join(os.getcwd(), 'datafiles', 'BD_71_train.csv'),header =None)
df_train['real_label'] = df_train[0].apply(lambda x: x.split('/')[-2])
df_train['img_name'] = df_train[0].apply(lambda x: x.split('/')[-1])
df_train['txt_path'] = df_train['img_name'].apply(lambda x: img_name_to_txt_path('train', x))


for i in range(len(df_train)):

    boxes = get_json_annotations(df_train['img_name'][i], df_train['real_label'][i])
    txt_path = df_train['txt_path'][i]
    
    write_txt_file(boxes, txt_path)

## 2.3 Val

In [159]:
df_val = pd.read_csv(os.path.join(os.getcwd(), 'datafiles', 'BD_71_val.csv'),header =None)
df_val['real_label'] = df_val[0].apply(lambda x: x.split('/')[-2])
df_val['img_name'] = df_val[0].apply(lambda x: x.split('/')[-1])
df_val['txt_path'] = df_val['img_name'].apply(lambda x: img_name_to_txt_path('val', x))


for i in range(len(df_val)):

    boxes = get_json_annotations(df_val['img_name'][i], df_val['real_label'][i])
    txt_path = df_val['txt_path'][i]
    
    write_txt_file(boxes, txt_path)

# 2 : FIX  PROBLEM WITH TXT FILES

stored as 'img..txt' instead of 'img.txt'

In [10]:
def rename_txt_files(dataset):
    
    for root,dirs,files in os.walk(dataset):

        for file in files:

            if file.endswith('.txt'):

                txt_path = os.path.join(root, file)
                new_txt_path = txt_path.replace('..txt', '.txt')
                os.rename(txt_path, new_txt_path)


rename_txt_files('./datasets/BD_71_copy')


In [11]:
rename_txt_files('./datasets/BD_71')

# 3. Reshape with copy

In [1]:


import os 
path_db_copy = os.path.join('./datasets/BD_71_copy/')

# # images / labels
# for dir in os.listdir(path_db_copy):

#     # train / test / val
#     path_dir = os.path.join(path_db_copy, dir)


#     for sub_dir in os.listdir(path_dir):

#         for ss_dir in os.listdir(os.path.join(path_dir, sub_dir)):

#             for file in os.listdir(os.path.join(path_dir, sub_dir, ss_dir)):

#                 file_path = os.path.join(path_dir, sub_dir, ss_dir, file)

#                 # move file to previous dir
#                 cmd = 'mv "' + os.path.join(path_dir, sub_dir, ss_dir, file) + '" "' + os.path.join(path_dir, sub_dir, file) + '"'
#                 os.system(cmd)

 


NotADirectoryError: [Errno 20] Not a directory: './datasets/BD_71_copy/labels/train/7_avril_023x..txt'

In [5]:
path_db_copy = os.path.join('./datasets/BD_71_copy/')

for root, dirs, files in os.walk(path_db_copy):
    
    for file in files:
        
        old_path = os.path.join(root, file)

        # move file to previous dir
        new_path = os.path.join(root, '..', file)

        cmd = 'mv "' + old_path + '" "' + new_path + '"'
        os.system(cmd)

In [6]:
for root,dirs,files in os.walk(path_db_copy):
    
    for dir in dirs:
        
        # remoe empty dir
        cmd = 'rm -r "' + os.path.join(root, dir) + '"'

# 4. Create a sample of the dataset to test the code

In [3]:
! mkdir datasets/BD_71_sample

! mkdir datasets/BD_71_sample/images
! mkdir datasets/BD_71_sample/labels

! mkdir datasets/BD_71_sample/images/train
! mkdir datasets/BD_71_sample/images/test
! mkdir datasets/BD_71_sample/images/valid

! mkdir datasets/BD_71_sample/labels/train
! mkdir datasets/BD_71_sample/labels/test
! mkdir datasets/BD_71_sample/labels/valid

In [4]:
# select 8 images from BD_71_copy train 2 from test and 2 from val

import os
import random

path_db_copy = os.path.join('./datasets/BD_71_copy/')
path_db_sample = os.path.join('./datasets/BD_71_sample/')

def sample_images(path_db_copy, split, n):

    
    dir = os.path.join(path_db_copy,'images', split)
    file = os.listdir(dir)
    file_paths = [os.path.join(dir, x) for x in file]

    # shuffle files_path
    random.shuffle(file_paths)

    # select n first files
    file_paths = file_paths[:n]

    extensions = [x.split('.')[-1] for x in file_paths]

    txt_paths = [x.replace(extensions[i], 'txt') for i,x in enumerate(file_paths)]
    txt_paths = [x.replace('images', 'labels') for x in txt_paths]

    new_file_paths = [x.replace(path_db_copy, path_db_sample) for x in file_paths]
    new_txt_paths = [x.replace(path_db_copy, path_db_sample) for x in txt_paths]

    # copy files
    for i in range(len(file_paths)):

        cmd = 'cp "' + file_paths[i] + '" "' + new_file_paths[i] + '"'
        os.system(cmd)
        


        cmd = 'cp "' + txt_paths[i] + '" "' + new_txt_paths[i] + '"'
        os.system(cmd)
        

    

In [5]:
sample_images(path_db_copy, 'train', 80)
sample_images(path_db_copy, 'test', 20)
sample_images(path_db_copy, 'valid', 20)

In [2]:
! rm -r datasets/BD_71_sample